In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [ ]:
#creates analysis/exploration.json

exploration = rs.explore_data(
    data=data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [ ]:
# fits model

model = rs.glm(
    formula="ClaimCount ~ VehPower + VehAge + C(Area) + C(Region)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [ ]:
model = rs.glm(
    formula="ClaimCount ~ C(Area) + bs(VehAge, df=4)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [2]:

model = rs.glm(
    formula="ClaimCount ~ ns(VehAge, df=4) + DrivAge + TE(Region) + TE(VehBrand) + TE(VehGas)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()

In [3]:
print(model.summary())

                                 GLM Results                                  

Family:              NegativeBinomial No. Observations:        678012
Link Function:       (default)       Df Residuals:            678004
Method:              IRLS + Ridge    Df Model:                     7
Scale:               0.0518          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                  12
Non-zero coefs:      7              

Log-Likelihood:         -145895.5792 Deviance:                 35124.0212
AIC:                     291807.1585 Null Deviance:           186671.5709
BIC:                     291898.5738 Pearson chi2:             1137521.36
Converged:           True           

------------------------------------------------------------------------------
Variable               Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept           -1.5203     0

In [4]:
#writes analysis/diagnostics
model.diagnostics(
    data=data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(VehAge, df=4) + DrivAge + TE(Region) + TE(VehBrand) + TE(VehGas)', 'family': 'NegativeBinomial(theta=0.0100)', 'link': 'log', 'n_observations': 678012, 'n_parameters': 8, 'degrees_of_freedom_residual': 678004, 'converged': True, 'iterations': 12}, convergence_details=ConvergenceDetails(max_iterations_allowed=25, iterations_used=12, converged=True, reason='converged'), fit_statistics={'deviance': 35124.021156029485, 'null_deviance': 33408.850174904575, 'deviance_explained': -0.051338821065242124, 'log_likelihood': -177312.27099037162, 'aic': 354640.54198074323, 'bic': 354731.9573428703, 'pearson_chi2': 592575.9067763546, 'dispersion_deviance': 0.05180503530367001, 'dispersion_pearson': 0.8740006058612555}, loss_metrics={'family_deviance_loss': 0.05180442404563372, 'mse': 288949427991443.56, 'mae': 20645.457267695252, 'rmse': 16998512.522907514}, calibration={'ae_ratio': 0.0, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, '

In [ ]:
model = rs.glm(
    formula="ClaimCount ~ VehAge + bs(DrivAge, df=4) + TE(Region) + TE(VehBrand) + TE(VehGas) + TE(Area) + TE(Area):bs(DrivAge, df=4)",
    data=data,
    family="negbinomial",
    offset="Exposure"
).fit()


thread '<unnamed>' panicked at crates/rustystats-core/src/formula/mod.rs:97:21:
Unknown argument 'df' in TE(). Supported: prior_weight (pw), n_permutations (nperm)
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: Unknown argument 'df' in TE(). Supported: prior_weight (pw), n_permutations (nperm)